# Lab 2: SQL Practice

Find out who were the top 3 most purchased (by revenue) artist in California (CA) each year. 

![Schema](https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)


In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('../data/chinook.db')

In [2]:
# The only external resource I used was the RANK() window function tutorial here:
# https://www.sqlitetutorial.net/sqlite-window-functions/sqlite-rank/

df = pd.read_sql_query("""WITH main AS (
              SELECT a.UnitPrice, a.Quantity, b.AlbumId, c.ArtistId, d.Name, e.total, strftime('%Y', e.InvoiceDate) as Year, f.State
              FROM invoice_items a INNER JOIN tracks b ON a.TrackId = b.TrackId  
              INNER JOIN albums c ON b.AlbumId = c.AlbumId 
              INNER JOIN artists d ON c.ArtistId = d.ArtistId 
              INNER JOIN invoices e ON a.invoiceId = e.invoiceId 
              INNER JOIN customers f ON e.CustomerId = f.CustomerId 
              WHERE f.State == 'CA' ),
    grouped_table AS (
    SELECT Name, Year, State, SUM(total) as Revenue FROM main GROUP BY Name, Year
    ORDER BY Year,Revenue DESC ),
    ranked_table AS (SELECT *,
                    RANK () OVER (PARTITION BY Year ORDER BY Revenue DESC) RevRank
                    FROM grouped_table)
    SELECT * FROM ranked_table WHERE RevRank <= 3
    """,conn)

In [3]:
df

,Name,Year,State,Revenue,RevRank
0,Deep Purple,2009,CA,55.44,1
1,The Cult,2009,CA,44.55,2
2,Djavan,2009,CA,41.58,3
3,Eric Clapton,2009,CA,41.58,3
4,Iron Maiden,2010,CA,194.04,1
5,Deep Purple,2010,CA,27.72,2
6,Ed Motta,2010,CA,27.72,2
7,Gene Krupa,2010,CA,27.72,2
8,Miles Davis,2010,CA,27.72,2
9,Toquinho & Vinícius,2010,CA,27.72,2


In [4]:
conn.close()

### Notes:

- Per the Hint, I decided to outuput all ties by creating a RANK() column.
- If the CEO wanted to give bonuses to tied artists, I recommend evenly splitting the remaining pool of funds (for example, in 2010: Iron Maiden would get the full 1st place bonus, then the six who tied for 2nd place would split the remaining money).
- I felt like I had to use INNER JOIN for this lab, we cannot get to the final table without having all the selected data. However a better practice might be to LEFT JOIN then filter out rows with missing data later.
- To obtain revenue, I used the method of selecting `total` from `invoices` table. However, it may also be reasonable to multiply `UnitPrice` by `Quantity` from the `invoice_items` table (but I think this will give a different answer).